In [1]:
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam, RMSprop
from torch.optim.lr_scheduler import OneCycleLR, StepLR, LinearLR
from data.preprocessing import *
from data.data_utils import *
from models.macro_architectures import *
from models.df_models import *
from models.utils import *

In [2]:
hidden_size = 100
architecture = (50,25,10)
num_layers = 6
sequence_length_hour = 96  #last 4 days
output_size = 1 #depends on target
dict_values = ['dst_kyoto', 'kp_gfz']
time_steps = 0 #after how time steps you want to inference

In [3]:
device = get_default_device()

In [4]:
arch = 'LSTM' #change depending on arch type
encoder = LSTM(input_size = 20 ,hidden_size =  hidden_size, num_layers = num_layers, bidirectional=True)
fc = DeepNeuralNetwork(2*hidden_size, output_size, *architecture)
model = to_device(Sing2Sing(encoder, fc), device)

In [5]:
#DATA PROCESSING
start_time = '20210101'
end_time = '20230802'
scrap_date = interval_time(start_time, end_time)
months = list(set([day[:6] for day in scrap_date]))
import_Dst(months)
l1_sample, l2_sample, dst, kp = automated_preprocessing(scrap_date, sep = False)
#l1_sample_hour = (l1_sample[0].resample('60min').mean(), l1_sample[1].resample('60min').mean()) #multhead
#l2_sample_hour = (l2_sample[0].resample('60min').mean(), l2_sample[1].resample('60min').mean()) #multhead encoder forcing
l1_sample = l1_sample.resample('60min').mean()
l2_sample = l2_sample.resample('60min').mean()

C:\Users\jorge\Desktop\SMFGF\SMFGF-SpaceApps\data\preprocessing.py:360: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  today_kp = kp[day][0:8]


In [6]:
hour_kp_dataset = MainToSingleTarget(l1_sample, dst, sequence_length_hour, pred_length, True, False, 'dst_kyoto', None, time_step_ahead = time_steps, multiclass = True, device = 'cuda')

In [7]:
#Test:5% training: 95%

test_size = round(0.05*len(hour_kp_dataset))

train_hour_kp, test_hour_kp = random_split(hour_kp_dataset , [len(hour_kp_dataset) - test_size, test_size])

batch_size = 512  #Change based on GPU capacity

train_hour_kp_dl = DataLoader(train_hour_kp, batch_size, shuffle=True, num_workers=4)
train_hour_kp_dl = DeviceDataLoader(train_hour_kp_dl, device)
test_hour_kp_dl = DataLoader(test_hour_kp, batch_size*2, num_workers=4)
test_hour_kp_dl = DeviceDataLoader(test_hour_kp_dl, device)

In [8]:
history = []

In [9]:
##hyperparameters
epochs = 10
max_lr = 6e-5
weigth_decay = 1e-6
grad_clip = 5e-4
opt_func = Adam
lr_sched = LinearLR
#lr_sched = LinearLR
start_factor = 1
end_factor = 0.01
steps = epochs
gamma = 0.999
weights = [0.1,0.1,1]
encoder_forcing = False
#opt_func = RMSprop

In [10]:
history += model.fit(epochs, max_lr, train_hour_kp_dl, test_hour_kp_dl, weigth_decay, grad_clip, opt_func, lr_sched, start_factor, end_factor, steps, gamma, weights, encoder_forcing)

RuntimeError: Found dtype Int but expected Float

In [ ]:
import os
os.makedirs('sing2sing', exist_ok=True)
torch.save(model, f'sing2sing/{hidden_size}_{num_layers}_{pred_length}_{time_steps}_{arch}.pt')
try:
    os.remove(f'sing2sing/{hidden_size}_{num_layers}_{pred_length}_{time_steps}_{arch}.csv')
except:
    pass
with open(f'sing2sing/{hidden_size}_{num_layers}_{pred_length}_{time_steps}_{arch}.csv', 'w') as file:
    file.write(','.join(map(str,list(history[-1].values()))))